<a href="https://colab.research.google.com/github/Saharsh2k5/ML_ASSIGNMENT_1/blob/main/Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1) Demonstrate how to use Zero-Shot Learning and Few-Shot Learning to classify human activities based on the featurized accelerometer data. Qualitatively demonstrate the performance of Few-Shot Learning with Zero-Shot Learning. Which method performs better? Why? [1 marks]

In [ ]:
pip install langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [ ]:
import os
import pandas as pd
import numpy as np
from langchain_groq.chat_models import ChatGroq
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

train_file_path = "/content/drive/MyDrive/ML Assignment1/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt"
features_file = "/content/drive/MyDrive/ML Assignment1/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/features.txt"
train_file_path_y = "/content/drive/MyDrive/ML Assignment1/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt"
test_file_path = "/content/drive/MyDrive/ML Assignment1/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt"
test_file_path_y = "/content/drive/MyDrive/ML Assignment1/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt"

def data_sorting(test_file_path,num_of_samples,test_file_path_y):
  df_x = pd.read_csv(test_file_path, delimiter='\s+', header=None)

  df_standardized_x = df_x.apply(lambda x: (x - x.mean()) / x.std(), axis=1)

  with open(features_file, 'r') as file:
      feature_names = [line.split()[1] for line in file]

  df_standardized_x.columns = feature_names

  X_array = df_standardized_x.to_numpy()

  pca = PCA(n_components=2)
  X_pca = pca.fit_transform(X_array)

  df_pca = pd.DataFrame(data=X_pca)

  print("Original shape:", X_array.shape)
  print("Reduced shape:", df_pca.shape)


  df_y = pd.read_csv(test_file_path_y, delimiter='\s+', header=None)

  df_pca_sampled = df_pca.sample(n=num_of_samples, random_state=0)  # You can set random_state for reproducibility

  sampled_indices = df_pca_sampled.index

  df_y_sampled = df_y.loc[sampled_indices]

  X_train = df_pca_sampled
  y_train = df_y_sampled

  label_mapping = {
    1.0: "WALKING",
    2.0: "WALKING_UPSTAIRS",
    3.0: "WALKING_DOWNSTAIRS",
    4.0: "SITTING",
    5.0: "STANDING",
    6.0: "LAYING"}

  y_train = y_train[0].map(lambda x: label_mapping.get(x, "UNKNOWN"))

  y_train = y_train.astype(str)

  return X_train,y_train

model_name = "llama3-70b"
groq_models = {
    "llama3-70b": "llama3-70b-8192",
    "mixtral": "mixtral-8x7b-32768",
    "gemma-7b": "gemma-7b-it",
    "llama3.1-70b": "llama-3.1-70b-versatile",
    "llama3-8b": "llama3-8b-8192",
    "llama3.1-8b": "llama-3.1-8b-instant",
    "gemma-9b": "gemma2-9b-it"
}
import json

with open('/content/drive/MyDrive/key.json') as config_file:
    config = json.load(config_file)
    Groq_Token = config['groq_token']

llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)


In [ ]:
#ZERO SHOT LEARNING

X_test_split,y_test_split=data_sorting(train_file_path,10,train_file_path_y)

label_set = ["LAYING", "SITTING", "STANDING", "WALKING", "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS"]

def classify_sample(sample):
    features = ", ".join([f"Feature_{i+1}: {value}" for i, value in enumerate(sample)])
    query = f"""
    * You are a human activities classification model.
    * Your task is to analyze the given features and classify them as one of these 6 activities: {', '.join(label_set)}.
    * Provide only the activity label.

    Features:
    {features}
    """
    answer = llm.invoke(query)
    return answer.content.strip()


def classify_samples(X):
    predictions = []
    for _, row in X.iterrows():
        prediction = classify_sample(row)
        predictions.append(prediction)
    return pd.Series(predictions)

test_predictions = classify_samples(X_test_split)

true_test_labels = np.array(y_test_split.astype(str))

test_predictions = np.array(test_predictions.astype(str))

test_accuracy = accuracy_score(true_test_labels, test_predictions)
print("Zero Shot Testing Accuracy:", test_accuracy)

Original shape: (7352, 561)
Reduced shape: (7352, 2)
Zero Shot Testing Accuracy: 0.2


In [ ]:
#FEW SHOT LEARNING
X_train,y_train=data_sorting(train_file_path,20,train_file_path_y)
X_test_split,y_test_split=data_sorting(test_file_path,10,test_file_path_y)

feature_label_dict = {}
for i in X_train.index:
    features = tuple(X_train.loc[i])
    label = y_train.loc[i]
    feature_label_dict[features] = label

def classify_sample(sample,feature_label_dict):
    features = ", ".join([f"Feature_{i+1}: {value}" for i, value in enumerate(sample)])
    query = f"""
    * You are a human activities classification model.
    * Your task is to analyze the given features and classify them as one of these 6 activities: {', '.join(label_set)}.
    * Provide only the activity label.

    Here are few examples:
    {feature_label_dict}

    Features:
    {features}
    """
    answer = llm.invoke(query)
    return answer.content.strip()

def classify_samples(X,feature_label_dict):
    predictions = []
    for _, row in X.iterrows():
        prediction = classify_sample(row,feature_label_dict)
        predictions.append(prediction)
    return pd.Series(predictions)

test_predictions = classify_samples(X_test_split,feature_label_dict)

true_test_labels = np.array(y_test_split.astype(str))

test_predictions = np.array(test_predictions.astype(str))

print("True Test Labels:", true_test_labels)
print("Test Predictions:", test_predictions)

test_accuracy = accuracy_score(true_test_labels, test_predictions)
print("Few Shot Testing Accuracy: ",test_accuracy)

Original shape: (7352, 561)
Reduced shape: (7352, 2)
Original shape: (2947, 561)
Reduced shape: (2947, 2)
True Test Labels: ['WALKING' 'LAYING' 'LAYING' 'WALKING_UPSTAIRS' 'WALKING' 'LAYING'
 'WALKING_UPSTAIRS' 'LAYING' 'WALKING' 'STANDING']
Test Predictions: ['WALKING' 'SITTING' 'LAYING' 'WALKING' 'WALKING' 'LAYING' 'WALKING'
 'LAYING' 'WALKING' 'STANDING']
Few Shot Testing Accuracy:  0.7


Few-shot learning performs better than zero-shot learning since the features of accelerometer data are extracted well, such as walking, standing, and so on. So, the model of few-shot learning can perform better as there are a sufficient number of examples per class.

2) Quantitatively compare the accuracy of Few-Shot Learning with Decision Trees (You may use a subset of the test set if you encounter rate-limiting issues). Which method performs better? Why? [1 marks]

In [ ]:
#Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X_train,y_train=data_sorting(train_file_path,30,train_file_path_y)
X_test,y_test=data_sorting(test_file_path,10,test_file_path_y)

clf = DecisionTreeClassifier(max_depth=4,random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test_split)

accuracy = accuracy_score(y_test_split, y_pred)
print('Decision Tree Accuracy on test data: ',accuracy)

Original shape: (7352, 561)
Reduced shape: (7352, 2)
Original shape: (2947, 561)
Reduced shape: (2947, 2)
Decision Tree Accuracy on test data:  0.7


if the data we are taking is large and well structured then the decision tree classifier performs better whereas if the data is of limited samples then the few shot learning is useful. If we are specifically speaking which is better then the method depends on the depth of the decision tree. A decision tree of optimal depth might be better than few shot learning while a decision tree which is underfitting or overfitting might give lower accuracy than few shot learning. As we can see the results we have taken the minimised random data in which few shot learning performs better than decision trees

3) What are the limitations of Zero-Shot Learning and Few-Shot Learning in the context of classifying human activities based on featurized accelerometer data? [1 marks]

Limitations of Zero-shot learning:

If the descriptions or attributes for the data are vague or not well-defined, the model might struggle to make accurate predictions for unseen activities.

Human activities can vary widely, and if unseen activities are fundamentally different from those seen during training, the model might perform poorly.

Human activity data can be highly variable due to differences in individual behavior, context, and sensor placement. ZSL might struggle to generalize across such variability without adequate attribute representation.

Limitations of Few-shot learning:

Few-Shot Learning aims to classify new activities with very few examples. If the number of examples is too small, the model might not capture enough features to perform accurately.

With few examples, there is a risk of overfitting where the model may learn to memorize the few samples rather than generalizing from them.

Human activities can have subtle differences and may be very similar to each other. Few-Shot Learning models may find it challenging to differentiate between activities that are highly similar, especially with limited examples.

4) What does the model classify when given input from an entirely new activity that it hasn't seen before? [0.5 mark]

In zero-shot and few-shot learning models, the classification behavior for entirely new activities can differ:

Zero-Shot Learning: The model attempts to classify the new activity based on its understanding of the general features and relationships between the known activities. If the new activity shares similarities with the known ones, the model might try to categorize it into the closest matching known class or indicate that it is not part of the known classes.

Few-Shot Learning: The model might require a few examples of the new activity to classify it accurately. If it has been given a few examples of the new activity, it can use those examples to understand and classify new instances of the same activity. Without any examples, it might classify it as one of the known classes or indicate that it does not fit well with any of them.

In both cases, the model's ability to handle entirely new activities depends on how well it can generalize from the training data or how effectively it can leverage any limited examples provided.

5) Test the model with random data (ensuring the data has the same dimensions and range as the previous input) and report the results. [0.5 mark]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from langchain_groq.chat_models import ChatGroq

X_train,y_train=data_sorting(train_file_path,20,train_file_path_y)

label_set = ["LAYING", "SITTING", "STANDING", "WALKING", "WALKING_UPSTAIRS", "WALKING_DOWNSTAIRS"]

def classify_sample(sample, feature_label_dict):
    features = ", ".join([f"Feature_{i+1}: {value}" for i, value in enumerate(sample)])
    query = f"""
    * You are a human activities classification model.
    * Your task is to analyze the given features and classify them as one of these 6 activities: {', '.join(label_set)}.
    * Provide only the activity label.

    Here are few examples:
    {feature_label_dict}

    Features:
    {features}
    """
    answer = llm.invoke(query)
    return answer.content.strip()

def classify_samples(X, feature_label_dict):
    predictions = []
    for _, row in X.iterrows():
        prediction = classify_sample(row, feature_label_dict)
        predictions.append(prediction)
    return pd.Series(predictions)

feature_label_dict = {tuple(row): label for idx, row in X_train.iterrows() for label in [y_train.loc[idx]]}

def generate_random_data(rows, cols):
    return pd.DataFrame(np.random.rand(rows, cols), columns=[f"Feature_{i+1}" for i in range(cols)])

random_X_train = generate_random_data(X_train.shape[0], X_train.shape[1])

random_train_predictions = classify_samples(random_X_train, feature_label_dict)

print("Random Train Predictions:", random_train_predictions)




Original shape: (7352, 561)
Reduced shape: (7352, 2)
Random Train Predictions: 0     STANDING
1     STANDING
2     STANDING
3     STANDING
4     STANDING
5     STANDING
6     STANDING
7     STANDING
8     STANDING
9     STANDING
10    STANDING
11    STANDING
12    STANDING
13    STANDING
14     WALKING
15    STANDING
16    STANDING
17    STANDING
18    STANDING
19    STANDING
dtype: object
